# Quantum Traveling Salesman Problem Solver

This notebook implements a quantum solution for the Traveling Salesman Problem (TSP) using Qiskit.
The TSP is about finding the shortest possible route that visits each city exactly once and returns to the origin city.

We'll use the Quantum Approximate Optimization Algorithm (QAOA) approach.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

from qiskit.visualization import plot_histogram

## Problem Setup

First, we'll create a small example problem with 4 cities.
The distances between cities are represented using an adjacency matrix.

In [ ]:
# Define cities and their distances
number_of_cities = 4

# Create a random distance matrix (symmetric)
# distances[i][j] represents the distance between city i and city j
np.random.seed(42)  # For reproducibility
distances = np.random.rand(number_of_cities, number_of_cities)
distances = (distances + distances.T) / 2  # Make symmetric
np.fill_diagonal(distances, 0)  # Distance to same city is 0

# Visualize the cities as a graph
graph = nx.from_numpy_array(distances)
pos = nx.spring_layout(graph)
nx.draw(graph, pos, with_labels=True)
labels = nx.get_edge_attributes(graph, 'weight')
nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
plt.show()

## Quantum Circuit Implementation

We'll encode the TSP into a Quantum Approximate Optimization Algorithm (QAOA) problem.
The encoding uses binary variables x_{i,p} where:
- i represents the city
- p represents the position in the tour

Constraints:
1. Each city must be visited exactly once
2. Each position must be used exactly once
3. The total distance should be minimized

In [ ]:
def create_mixer_hamiltonian(quantum_circuit, qubits):
    """Apply mixer Hamiltonian (X rotations) to create superposition."""
    for qubit in qubits:
        quantum_circuit.rx(np.pi, qubit)

def create_cost_hamiltonian(quantum_circuit, distances, qubits, gamma):
    """Apply cost Hamiltonian using ZZ interactions between cities."""
    n = int(np.sqrt(len(qubits)))  # number of cities
    for i in range(n):
        for j in range(i+1, n):
            for pos in range(n):
                q1 = i * n + pos
                q2 = j * n + ((pos + 1) % n)
                # Apply ZZ interaction weighted by distance
                quantum_circuit.cx(qubits[q1], qubits[q2])
                quantum_circuit.rz(2 * gamma * distances[i,j], qubits[q2])
                quantum_circuit.cx(qubits[q1], qubits[q2])

def create_tsp_qaoa_circuit(distances, p=1):
    """Create QAOA circuit for TSP with p layers."""
    n = len(distances)
    num_qubits = n * n
    qubits = range(num_qubits)
    
    # Initialize parameters
    betas = np.random.uniform(0, np.pi, p)
    gammas = np.random.uniform(0, np.pi, p)
    
    qc = QuantumCircuit(num_qubits)
    
    # Initial state: equal superposition
    qc.h(qubits)
    
    # QAOA layers
    for layer in range(p):
        # Cost Hamiltonian
        create_cost_hamiltonian(qc, distances, qubits, gammas[layer])
        
        # Mixer Hamiltonian
        create_mixer_hamiltonian(qc, qubits)
        
        # Add position constraints
        for i in range(n):
            for pos in range(n):
                q1 = i * n + pos
                q2 = i * n + ((pos + 1) % n)
                qc.cx(q1, q2)
    
    # Measure all qubits
    qc.measure_all()
    
    return qc
    
create_tsp_qaoa_circuit(distances, p=1).draw('mpl')

In [ ]:
# Create and run the improved QAOA circuit
improved_circuit = create_tsp_qaoa_circuit(distances)

# Run on the quantum simulator
job = quantum_instance.run(improved_circuit)
result = job.result()

# Process results
def calculate_tour_cost(tour, distances):
    """Calculate the total cost of a tour."""
    cost = 0
    n = len(tour)
    for i in range(n):
        cost += distances[tour[i]][tour[(i + 1) % n]]
    return cost

def is_valid_tour(tour, n):
    """Check if the tour is valid (visits each city exactly once)."""
    return len(tour) == n and len(set(tour)) == n

# Analyze and display results
counts = result.get_counts()
best_cost = float('inf')
best_tour = None

for bitstring in counts:
    tour = interpret_bitstring(bitstring, number_of_cities)
    if tour and is_valid_tour(tour, number_of_cities):
        cost = calculate_tour_cost(tour, distances)
        if cost < best_cost:
            best_cost = cost
            best_tour = tour

print(f"Best tour found: {best_tour}")
print(f"Tour cost: {best_cost:.2f}")

# Visualize the best tour
if best_tour is not None:
    plt.figure(figsize=(8, 6))
    G = nx.Graph()
    for i in range(len(best_tour)):
        current = best_tour[i]
        next_city = best_tour[(i + 1) % len(best_tour)]
        G.add_edge(current, next_city)
    
    nx.draw(G, pos, with_labels=True, node_color='lightblue', 
           node_size=500, font_size=16, font_weight='bold')
    plt.title('Best TSP Tour Found')
    plt.show()

## Execute the Quantum Circuit

Now we'll run the circuit and analyze the results.
Note: This is a simplified implementation. A full TSP solution would require:
1. More sophisticated qubit encoding
2. Additional constraints for valid tours
3. Problem-specific mixing operators
4. Post-processing to ensure valid solutions

In [ ]:
# Execute the circuit
job = execute(quantum_circuit, quantum_backend, shots=1000)
result = job.result()

# Get the counts of measurement results
counts = result.get_counts(quantum_circuit)

# Post-process the results
def interpret_bitstring(bitstring, number_of_cities):
    """
    Convert a measurement bitstring into a possible tour.
    This is a simplified interpretation - a full implementation would need more sophisticated decoding.
    """
    # Reshape the bitstring into a matrix
    bits = [int(bit) for bit in bitstring]
    matrix = np.array(bits).reshape(number_of_cities, number_of_cities)
    
    # Try to extract a valid tour (simplified)
    tour = []
    for position in range(number_of_cities):
        city_bits = matrix[:, position]
        if 1 in city_bits:
            tour.append(np.where(city_bits == 1)[0][0])
    return tour if len(tour) == number_of_cities else None

# Analyze results
print("\nPossible Tours:")
valid_tours = 0
for bitstring, count in counts.items():
    tour = interpret_bitstring(bitstring, number_of_cities)
    if tour is not None:
        # Calculate tour distance
        tour_distance = sum(distances[tour[i]][tour[(i+1)%number_of_cities]] 
                          for i in range(number_of_cities))
        print(f"Tour {tour}: Distance {tour_distance:.2f} (Found {count} times)")
        valid_tours += count

print(f"\nFound {valid_tours} valid tours out of 1000 measurements")

# Note: Due to the simplified implementation, many measurements
# might not correspond to valid tours. A full implementation would
# use additional quantum gates and constraints to ensure valid solutions.